# Indivudal User Stats

## Imports

In [20]:
import pandas as pd
import plotly.express as px
import re

## Import CSV

In [3]:
df = pd.read_csv('./extraction/emissions.csv')
df.head()

,address,red_meat,controller,grains,action,endpoint,dairy,cellphone,tv,computer,...,items/total,red_meat_co2_emissions,grains_co2_emissions,dairy_co2_emissions,cellphone_co2_emissions,tv_co2_emissions,computer_co2_emissions,car_co2_emissions,walking_co2_emissions,public_transport_co2_emissions
0,178373,100 pounds,LocationController,2 pounds,Get,(OWL) https://uwo.ca/edge/software,200 pounds,55 hours,43 hours,130 hours,...,2/1,800.0,7.4,1260.0,198.0,292.4,546.0,975.0,0,43.0
1,156354,89 pounds,Controlmite,167 pounds,Get,(OWL) https://uwo.ca/edge/software,42 pounds,42 hours,75 hours,88 hours,...,3/1,712.0,617.9,264.6,151.2,510.0,369.6,487.5,0,184.9
2,919323,26 pounds,ControlPublic,78 pounds,Get,https://uwo.ca/edge/electrical,78 pounds,100 hours,50 hours,75 hours,...,3/5,208.0,288.6,491.4,360.0,340.0,315.0,546.0,0,335.4
3,983575,35 pounds,Controlz,78 pounds,Get,(OWL) https://uwo.ca/edge/mechanical,64 pounds,78 hours,50 hours,5 hours,...,3/1,280.0,288.6,403.2,280.8,340.0,21.0,130.0,0,146.2
4,919325,55 pounds,LocationController,44 pounds,Get,https://uwo.ca/edge/mechanical,88 pounds,0 hours,100 hours,50 hours,...,3/3,440.0,162.8,554.4,0.0,680.0,210.0,507.0,0,94.6


## Graph

In [7]:
# List of tracked columns
emission_columns = [
    'red_meat_co2_emissions',
    'grains_co2_emissions',
    'dairy_co2_emissions',
    'cellphone_co2_emissions',
    'tv_co2_emissions', 
    'computer_co2_emissions',
    'car_co2_emissions',
    'walking_co2_emissions',
    'public_transport_co2_emissions'
]

In [11]:
# Get user info
username = input("Please enter your username: ")
values = df.loc[df['username'] == username]
user_values = [values.iloc[0][k] for k in emission_columns] # Get the values of the column
user_values

[800.0, 7.4, 1260.0, 198.0, 292.4, 546.0, 975.0, 0, 43.0]

In [25]:
# print the bar chart
fig = px.bar(
    x=[re.search("^(.*)_co2_emissions$", col).group(1) for col in emission_columns], # formatting the columns
    y=user_values,
    title=f"{username}'s CO2 emissions from the past year", # appl the title
    labels={ # apply labels
        "x": "form of emission", 
        "y": "kgs of CO2",
    },
)
fig.update_xaxes(categoryorder="total ascending") # convert axaxes to be ascending
fig.show()